In [ ]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
# %pip install pandas
# abfss://Data@onelake.dfs.fabric.microsoft.com/Datasets.Lakehouse/Files/booksdata.csv
# Files/booksdata.csv

- **CLEAN CSV**

In [ ]:
from pyspark.sql.functions import udf, col
from pyspark.sql.types import BooleanType
import re

# Read the CSV file into a Spark DataFrame
df = spark.read.format("csv").option("header", "true").load("abfss://Data@onelake.dfs.fabric.microsoft.com/Datasets.Lakehouse/Files/books.csv")
display(df)

# Define the UDF to check for non-ASCII characters
def has_non_ascii(value):
    if value is None:
        return False
    return bool(re.search(r'[^\x00-\x7F]', value))

# Register the UDF
has_non_ascii_udf = udf(has_non_ascii, BooleanType())

# Apply the UDF to each column that needs to be checked
df_clean = df.filter(~has_non_ascii_udf(col("Book")) & 
                     ~has_non_ascii_udf(col("Author")) & 
                     ~has_non_ascii_udf(col("Genres")))

# Display the cleaned DataFrame
display(df_clean)

# Coalesce the DataFrame to a single partition
df_clean_coalesced = df_clean.coalesce(1)

# Save the coalesced DataFrame to a new CSV file in Azure Data Lake Storage
cleaned_file_path = "abfss://Data@onelake.dfs.fabric.microsoft.com/Datasets.Lakehouse/Files/booksclean"
df_clean_coalesced.write.format("csv").option("header", "true").mode("overwrite").save(cleaned_file_path)
